In [5]:
# All imports, setup logging and get configuration file

from utils import get_configuration_file, setup_logging
import logging
import pandas as pd

setup_logging()
config = get_configuration_file()

/var/folders/t2/rv4p8zx13gj6mt8wcy_ckn3w0000gn/T/ipykernel_15786/2649509067.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [64]:
holiday_events = pd.read_csv(config["holiday_events_data_location"])
oil = pd.read_csv(config["oil_data_location"])
sample_submission = pd.read_csv(config["sample_submission_data_location"])
stores = pd.read_csv(config["stores_data_location"])
test = pd.read_csv(config["test_data_location"])
train = pd.read_csv(config["train_data_location"])
transactions = pd.read_csv(config["transactions_data_location"])

In [65]:
stores.rename(columns={"type":"store_type"}, inplace = True)
holiday_events.rename(columns={"type":"holiday_type"}, inplace = True)

In [66]:
train[["date","store_nbr","family"]].value_counts().unstack().max()

family
AUTOMOTIVE                    1
BABY CARE                     1
BEAUTY                        1
BEVERAGES                     1
BOOKS                         1
BREAD/BAKERY                  1
CELEBRATION                   1
CLEANING                      1
DAIRY                         1
DELI                          1
EGGS                          1
FROZEN FOODS                  1
GROCERY I                     1
GROCERY II                    1
HARDWARE                      1
HOME AND KITCHEN I            1
HOME AND KITCHEN II           1
HOME APPLIANCES               1
HOME CARE                     1
LADIESWEAR                    1
LAWN AND GARDEN               1
LINGERIE                      1
LIQUOR,WINE,BEER              1
MAGAZINES                     1
MEATS                         1
PERSONAL CARE                 1
PET SUPPLIES                  1
PLAYERS AND ELECTRONICS       1
POULTRY                       1
PREPARED FOODS                1
PRODUCE                       1
S

In [67]:
training_dataset = train.merge(stores, on = "store_nbr", how = "inner")
training_dataset = training_dataset.merge(oil, on = "date", how = "left")
# training_dataset = training_dataset.merge(holiday_events, on = "date", how = "left")
training_dataset

,id,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.57
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.57
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.57
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.57


In [75]:
a = holiday_events["date"].value_counts()
a = a[a>1]
a.shape


(31,)

In [69]:
holiday_events[holiday_events["date"] == "2014-06-25"]

,date,holiday_type,locale,locale_name,description,transferred
110,2014-06-25,Holiday,Local,Latacunga,Cantonizacion de Latacunga,False
111,2014-06-25,Holiday,Local,Machala,Fundacion de Machala,False
112,2014-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
113,2014-06-25,Event,National,Ecuador,Mundial de futbol Brasil: Ecuador-Francia,False


In [70]:
holiday_events["transferred"].value_counts()

transferred
False    338
True      12
Name: count, dtype: int64

In [71]:
holiday_events["holiday_type"].value_counts()

holiday_type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [72]:
holiday_events[holiday_events["date"] == "2012-10-09"]

,date,holiday_type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True


In [73]:
holiday_events[holiday_events["date"] == "2012-10-12"]

,date,holiday_type,locale,locale_name,description,transferred
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False


In [76]:
training_dataset["state"].value_counts()

state
Pichincha                         1055868
Guayas                             611292
Santo Domingo de los Tsachilas     166716
Azuay                              166716
Manabi                             166716
Cotopaxi                           111144
Tungurahua                         111144
Los Rios                           111144
El Oro                             111144
Chimborazo                          55572
Imbabura                            55572
Bolivar                             55572
Pastaza                             55572
Santa Elena                         55572
Loja                                55572
Esmeraldas                          55572
Name: count, dtype: int64